# Exporting Sentinel 2 Data With Correct Bands and Features for All of Costa Rica For a Given Year

In [ ]:
# Import Python API
# https://github.com/google/earthengine-api
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gbsc-gcp-lab-emordeca')

In [ ]:
# borders for costa rica
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
borders = ee.FeatureCollection('FAO/GAUL/2015/level0')
costa_rica = borders.filter(ee.Filter.eq('ADM0_NAME', 'Costa Rica'))
costa_rica_geo = costa_rica.geometry()

In [ ]:
# load in sentinel data, filter, and add bands

# Define a function to mask clouds
def maskS2clouds(image):
    qa = image.select('QA10')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    return image.updateMask(mask).divide(10000)

# Load Sentinel-2 data and apply cloud masking
# data source: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .filterBounds(costa_rica_geo) \
    .median() \
    .select(['B.*', 'TCI.*'])

def add_indices(image):
    # Add NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    image = image.addBands(ndvi)

    # Add SAVI
    savi = image.expression(
        '((NIR - Red) / (NIR + Red + 0.5)) * 1.5',
        {
            'NIR': image.select('B8'),
            'Red': image.select('B4')
        }
    ).rename('SAVI')
    image = image.addBands(savi)

    # Add NDMI
    ndmi = image.normalizedDifference(['B8', 'B11']).rename('NDMI')
    image = image.addBands(ndmi)

    # Add NDWI
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    image = image.addBands(ndwi)

    return image

# select bands of interest
bands = ["B2", "B3", "B4", "B8", "B11"]
dataset = dataset.select(bands)

# add indices
dataset = add_indices(dataset)
dataset = dataset.toFloat()


In [ ]:
# Export the image as a tif file via Google Earth Engine
export_params = {
  'scale': 30,  # 1 km resolution
  'region': costa_rica_geo,
  'fileFormat': 'GeoTIFF',  # You can change the file format as needed
  'maxPixels': 1e10
}

# Initiate the export task
task = ee.batch.Export.image.toDrive(
    image=dataset,
    description='costa_rica_sentinel_30m_2021_cs131',
    **export_params
)
task.start()